# HackDavis python library tests

## OCS-HackDavis Package Installation 

In [ ]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ ocs-hackdavis==0.18.0

## Import functions

In [ ]:
from ocs_hackdavis import (
    ucdavis_buildings,
    ucdavis_ceeds_of,
    ucdavis_dataview_id,
    ucdavis_config_data,
)

## List of buildings

In [ ]:
buildings = ucdavis_buildings()
len(buildings), buildings[:25]

## Function `ucdavis_ceeds_of` returns list of CEEDs for a given building

### Display them for all buildings

In [ ]:
for num, building in enumerate(ucdavis_buildings(), 1):
    print(f"[{num}] Building: {building} ==> CEEDS: {ucdavis_ceeds_of(building)}")

## Test error code 

### Bad argument of the right type

In [ ]:
# should fail
ucdavis_ceeds_of("Bad Building")

## Bad argument type

In [ ]:
# should fail - type error
ucdavis_ceeds_of(42)

## Get Data View ID of a given building+CEED pair

In [ ]:
dv_id = ucdavis_dataview_id("Activities and Recreation Center", "Electricity")
dv_id

## Import for OCS Sample Library and other necessary modules

In [ ]:
from ocs_sample_library_preview import OCSClient
import configparser
import io
import json
import pandas as pd

## Standard configuration file parsing and OCS client object 

#### Reference: https://github.com/osisoft/OSI-Samples-OCS/blob/master/basic_samples/DataViews/Python3/program.py#L150

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

namespace_id = config.get("Configurations", "Namespace")
print(f"namespace_id: '{namespace_id}'")

## Getting CSV data from Data View

In [ ]:
# Step 1) get data view id
dv_id = ucdavis_dataview_id("Activities and Recreation Center", "Electricity")

# Step 2) call data view
csv, token = ocs_client.DataViews.getDataInterpolated(
    namespace_id,
    dv_id,
    startIndex="2018-03-17",
    endIndex="2018-04-17",
    interval="00:20:00",
    count=250000,
    form="csvh",
)
# make sure there is no additional data
assert token is None

# display result subset
print(csv[:2048])

## Getting a Pandas dataframe from CSV

In [ ]:
# Step 3) transform CSV into a Pandas dataframe, convert Timestamp column to the right data type
df = pd.read_csv(io.StringIO(csv), parse_dates=['Timestamp'])
df

In [ ]:
# Same as above but keep only a subset of the columns
df_select = pd.read_csv(
    io.StringIO(csv),
    parse_dates=["Timestamp"],
    usecols=[
        "Timestamp",
        "AnnualUsage",
        "Cumulative Use",
        "Demand_kBtu",
        "Electricity_EUI",
        "MonthlyUsage",
    ],
)
df_select

## Access AF configuration data for a building+CEED pair

### Example

In [ ]:
ucdavis_config_data(ocs_client, namespace_id, "Activities and Recreation Center", "Electricity")

#### Note: each CEED of a building has a copy of the AF configuration data of the parent element (e.g. BuildingName, Latitude, Longitude, etc)

In [ ]:
ucdavis_config_data(ocs_client, namespace_id, "Activities and Recreation Center", "Steam")

## Display AF configuration data of 10 first building+CEEDs

In [ ]:
count = 1
for building in ucdavis_buildings():
    for ceed in ucdavis_ceeds_of(building):
        print(
            f"[{count}] {building} | {ceed}\n  {ucdavis_config_data(ocs_client, namespace_id, building, ceed)}"
        )
        count += 1
        print("------------------------")
    if count >= 10:
        break